In [1]:
from functions import*
import mplfinance as mpf

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Getting the data for stocks

In [2]:
df = pd.DataFrame()
df_alphavantage_365 = get_df_for_ticker_full('NFLX')
df_yfinance_365 = df.ta.ticker('NFLX', interval='1d')
df_yfinance = df.ta.ticker('NFLX', interval='1m')
df_alphavantage = get_intraday('NFLX', '1min')


ConnectionError: HTTPSConnectionPool(host='www.alphavantage.co', port=443): Max retries exceeded with url: /query?function=TIME_SERIES_DAILY&symbol=NFLX&outputsize=full&apikey=XIBTEAC0WWX5ONGP&datatype=json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000019798958550>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
df_yfinance['2023-08-08-15':'2023-08-08'].head()

In [ ]:
df_alphavantage['2023-08-08-15':'2023-08-08'].head()

In [ ]:
df_alphavantage_365['2023-06-01':'2023-07-01'].head()

In [ ]:
df_yfinance_365['2023-06-01':'2023-07-01'].head()

Both sources are similar on either intraday or daily charts, but continuing forward I will
be using the Alpha Vantage as I already wrote codes for them in the functions.py file.

Indicators from ta

In [ ]:
df = df_alphavantage_365.copy()

In [ ]:
df['rsi'] = df.ta.rsi()
df.columns = [x.lower() for x in df.columns]

In [ ]:
help(ta.rsi)

In [ ]:
df.head()

In [ ]:
#df = df.set_index(['date'])

In [ ]:
df.head()

In [ ]:
df.index[1]

In [ ]:
xd = df.copy()

In [ ]:
xd.head()

In [1]:
def strategy(df):
    trades = []
    current_trade = {'active': False, 'buy_price': 0, 'buy_time': None, 'profit': 0} 
    active = current_trade['active']
    for i in range(len(df) - 1):
        if df.iloc[i].rsi >= 70 and current_trade['active']:
            trades.append({
                'buy_price':current_trade['buy_price'],
                'buy_time':current_trade['buy_time'],
                'sell_price':df.iloc[i].close,
                'sell_time':df.index[i],
                'profit': df.iloc[i].close - current_trade['buy_price']
            })
            current_trade['active'] = False
        elif df.iloc[i].rsi <= 30 and not current_trade['active']:
            current_trade['buy_price'] = df.iloc[i].close
            current_trade['buy_time'] = df.index[i]
            current_trade['active'] = True

    
    trades = pd.DataFrame(trades) 
    return trades
    

In [ ]:
#xd[(xd.index.year == 2022) & (xd.index.month == 1)]

In [ ]:
temp = strategy(df)

In [ ]:
temp.head(20)

In [ ]:
df[(df.index.year == 2022)& ( df.index.month ==1)]

In [ ]:
def calc_profit(df):

    total_profit = profit_df['profit'].sum()
    print("Total Profit:", total_profit)

In [ ]:
temp['profit'].sum()


In [ ]:
dfjune = df.loc['2002-01-01' : '2004-01-01']

In [ ]:
tradesjune = temp.copy()

In [ ]:
df.tail()

In [ ]:
def is_rsi_increasing(args):
    return pd.Index(args).is_monotonic_increasing
       

In [ ]:
 is_rsi_increasing(1,22, 4)

In [ ]:
retard = df.copy()

In [ ]:
retard.head(16)

In [ ]:
retard['rsi_increasing'] = False
# Loop through the DataFrame and calculate the RSI trend
for i in range(len(retard) -1):
    rsi_values = retard['rsi'].iloc[i-2:i].values
    retard['rsi_increasing'].iloc[i] = is_rsi_increasing(rsi_values)
    print(rsi_values)





In [ ]:
df['rsi_increasing_new'] = df['rsi_increasing'].shift(-2)

In [ ]:
# Set 'buy_time' as the index for tradesjune dataframe
buys_df = tradesjune.set_index('buy_time').copy()

# Merge the dataframes based on index
merged_df = dfjune.merge(buys_df[['buy_price']], how='left', left_index=True, right_index=True)
# Rename the column to 'buy_price'
buys= pd.DataFrame({'close': merged_df['buy_price']})

# Set 'buy_time' as the index for tradesjune dataframe
sells_df = tradesjune.set_index('sell_time').copy()

# Merge the dataframes based on index
merged_df = dfjune.merge(sells_df[['sell_price']], how='left', left_index=True, right_index=True)
# Rename the column to 'buy_price'
sells= pd.DataFrame({'close': merged_df['sell_price']})


In [ ]:
buys.head(12)

In [ ]:
sells.head(12)

In [ ]:
kwargs = dict(title='Price chart', ylabel='Price' , ylabel_lower='Volume', type='line',volume=False,figratio=(16,9),figscale=1)

#https://github.com/matplotlib/mplfinance/blob/master/examples/styles.ipynb
RSI = mpf.make_addplot(dfjune.rsi, type='line', color='y', width=1, panel=1, title='RSI')
rsi_30_line = mpf.make_addplot([30] * len(dfjune), type='line', color='r', width=1, panel=1, secondary_y=False)
rsi_70_line = mpf.make_addplot([70] * len(dfjune), type='line', color='g', width=1, panel=1, secondary_y=False)
volume = mpf.make_addplot(dfjune.volume, type='bar', panel=2, color='black', secondary_y=False)


buy_arrows = mpf.make_addplot(buys['close'], scatter=True, markersize=100, color='g', marker='^', panel=0)
sell_arrows = mpf.make_addplot(sells['close'], scatter=True, markersize=100, color='r', marker='v', panel=0)
# Create marker addplot for trade execution points


mpf.plot(dfjune, **kwargs, main_panel=0, style='blueskies', addplot=[RSI, rsi_30_line, rsi_70_line, volume,buy_arrows,sell_arrows])



In [ ]:
sells.head()